In [1]:
# from dotenv import load_dotenv

# load_dotenv()

import pandas as pd
from typing import List
# from experiments.musique.inference_only import macro_averaging
from knowledge_propagation.utils import io, vars, extractor
import os
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
from scipy.stats import describe
from thefuzz import fuzz
from copy import deepcopy


from knowledge_propagation.modules.evaluators import (
    ExactMatchEvaluator,
    RougeEvaluator,
    OpenAIEvaluator,
)

from collections import defaultdict

llm_evaluator = OpenAIEvaluator()

def macro_averaging(df: pd.DataFrame, metrics: List[str], multi_level_averaging: List[str]):
    """
    Do macro-averaging over the given metrics and multi-level averaging categories.
    """
    extracted_multi_level_cols = [[m, "mean"] for m in metrics]
    while len(multi_level_averaging) > 0:
        # first take the mean over each generation,
        # and, only take `mean` of `rouge1` and  `llm_accuracy` column groups
        df_over_cols = df.groupby(multi_level_averaging, observed=True).describe()[extracted_multi_level_cols]
        # remove the multi-level column indices, since there's only one sub-level -- "mean"
        df_over_cols.columns = df_over_cols.columns.get_level_values(0)

        # reset index to flatten the multi-level column indices for the next macro-averaging class
        df = df_over_cols.reset_index(inplace=False)
        multi_level_averaging.pop(-1)
    return df
os.getcwd()

/home/zliu/miniconda3/envs/cpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/data/users/zliu/mend/notebooks'

In [39]:
df = pd.read_excel("/data/users/zliu/mend/mquake_exp_output/Llama-3.2-1B-eos-sft_clm-baseline_lr=1e-05_epoch=4.0_tuned-params=all/all_result_CF3k.xlsx")
print("Num of rows:", len(df))
df.describe()[["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "exact_match", "llm_accuracy",]].round(2) # 
pre_edit_df = df[df["stage"] == "pre-edit"]
post_edit_df = df[df["stage"] == "post-edit"]

Num of rows: 913


In [40]:
post_edit_df.groupby("question_tag").describe()

id                                         \
                               count       mean        std  min   25%   50%   
question_tag                                                                  
efficacy_multi-hop             450.0  74.500000  43.348500  0.0  37.0  74.5   
efficacy_single-hop-changed    391.0  73.603581  42.501691  0.0  38.0  71.0   
efficacy_single-hop-unchanged   72.0  73.805556  46.897142  0.0  34.0  72.0   

                                            predicted_answer_idx       ...  \
                                 75%    max                count mean  ...   
question_tag                                                           ...   
efficacy_multi-hop             112.0  149.0                450.0  0.0  ...   
efficacy_single-hop-changed    111.0  149.0                391.0  0.0  ...   
efficacy_single-hop-unchanged  111.0  149.0                 72.0  0.0  ...   

                              [Q][A] Acc PM           llm_accuracy            \
                                        75%       max        count      mean   
question_tag                                                                   
efficacy_multi-hop                 0.526316  0.818182        450.0  0.454667   
efficacy_single-hop-changed        0.736842  0.916667        391.0  0.769821   
efficacy_single-hop-unchanged      0.571429  0.666667         72.0  0.648611   

                                                                  
                                    std  min  25%  50%  75%  max  
question_tag                                                      
efficacy_multi-hop             0.432930  0.1  0.1  0.1  1.0  1.0  
efficacy_single-hop-changed    0.389029  0.1  0.3  1.0  1.0  1.0  
efficacy_single-hop-unchanged  0.437017  0.1  0.1  1.0  1.0  1.0  

[3 rows x 64 columns]

In [41]:
macro_averaging(post_edit_df, multi_level_averaging=["question_tag", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "llm_accuracy",]).round(2) # 

,question_tag,[A]|[Q] Acc EM,[A]|[Q] Acc PM,llm_accuracy
0,efficacy_multi-hop,0.34,0.72,0.45
1,efficacy_single-hop-changed,0.81,0.93,0.77
2,efficacy_single-hop-unchanged,0.43,0.78,0.65


In [21]:
# [["question", "predicted_answer", "answer"]]
for _, row in pre_edit_df[pre_edit_df["question_tag"] == "efficacy_single-hop"].iterrows():
    print(row["question"])
    print(row["predicted_answer"])
    print(row["answer"])
    print(row["llm_accuracy"])
    print() 

What type of music does Kim Junsu play?
KPOP
K-pop
1.0

Which country was K-pop created in?
South Korea
South Korea
1.0

What is the name of the current head of the South Korea government?
Park Geun-hye
Yoon Suk-yeol
0.1

What is the country of citizenship of Jussi Björling?
Finland
Sweden
0.1

What is the name of the current head of the Sweden government?
Fredrik Reinfeldt
Ulf Kristersson
0.1

Who is the developer of Chromecast?
Google
Google
1.0

Which city is the headquarter of Google located in?
Mountain View
Googleplex
1.0

Who is the original broadcaster of M*A*S*H?
Don Adams
CBS
0.1

Where was CBS founded?
New York
New York City
1.0

What is the name of the current head of the New York City government?
Michael Bloomberg
Eric Adams
0.1

Who is the employer of Larry Page?
GOOGLE
Google
1.0

Which city is the headquarter of Google located in?
Mountain View
Googleplex
1.0

Which city is the headquarter of Directors Guild of America located in?
Los Angeles
Los Angeles
1.0

What is th

In [38]:
subset = "CF3k"
save_dir = f"/data/users/zliu/mend/mquake_exp_output/Llama-3.2-1B-eos-sft_clm-baseline_lr=1e-05_epoch=4.0_tuned-params=all/individual_results_{subset}"
all_df = pd.concat([pd.read_excel(f) for f in glob(f"{save_dir}/*.xlsx", recursive=True)])
len(all_df)

all_df.to_excel(f"{save_dir}/../all_result_{subset}.xlsx", index=False)